# Data engineering capstone - Eleflow

#### Instalando dependências

In [129]:
!pip install pandas requests pendulum

#### imports necessários

In [130]:
import os
import json
import pendulum
import requests
import pandas as pd

ModuleNotFoundError: No module named 'pendulum'

## Carregar os dados de VRA
- Normalizar o cabeçalho para snake case
- Salvar estes dados

In [47]:
# Função para percorrer o diretório, pegar todos os arquivos e carregar as informações
def ler_jsons(diretorio):
    arquivos_json = [arquivo for arquivo in os.listdir(diretorio) if arquivo.endswith('.json')]

    for nome_arquivo in arquivos_json:
        caminho_arquivo = os.path.join(diretorio, nome_arquivo)
        with open(caminho_arquivo, 'r', encoding='utf-8-sig') as arquivo:
            yield json.load(arquivo)

In [48]:
# Nome das colunas em snake_case
nomes_colunas_vra = ['icao_empresa_aerea', 'numero_voo', 'codigo_autorizacao', 'codigo_tipo_linha',
                     'icao_aerodromo_origem', 'icao_aerodromo_destino', 'partida_prevista',
                     'partida_real', 'chegada_prevista', 'chegada_real', 'situacao_voo', 'codigo_justificativa']


In [49]:
# Diretório onde estão os arquivos JSON
diretorio_vra = './data_engineer_test/VRA'

In [50]:
# Criar um gerador
vra_generator = ler_jsons(diretorio_vra)

In [51]:
# Criação do dataframe com os dados de vra
vra_df = pd.DataFrame()

In [52]:
# Adicionando informações de cada arquivo JSON no dataframe
for lista_de_dicts in vra_generator:
    vra_df = vra_df._append(lista_de_dicts, ignore_index=True)

In [53]:
# Modificando nome das colunas do dataframe VRA
vra_df.columns = nomes_colunas_vra

In [122]:
print(vra_df)

       icao_empresa_aerea numero_voo codigo_autorizacao codigo_tipo_linha  \
0                     AZU       4478                  0                 N   
1                     AZU       4479                  0                 N   
2                     AZU       4479                  0                 N   
3                     AZU       4479                  0                 N   
4                     AZU       4479                  0                 N   
...                   ...        ...                ...               ...   
535798                PVV       8653                  9                 X   
535799                PVV       8752                  9                 X   
535800                PVV       8752                  9                 X   
535801                PVV       8752                  2                 I   
535802                PVG       9831                  9                 X   

       icao_aerodromo_origem icao_aerodromo_destino     partida_prevista  \

In [125]:
vra_df.to_csv('./data_export/VRA/vra_data.csv', index=False)

## Carregar dos dados de AIR_CIA
- Normalizar o cabeçalho para snake case
- Separar a coluna 'ICAO IATA' em duas colunas, seu conteúdo está separado por espaço e pode não conter o código IATA, caso não contenha o código IATA, deixe o valor nulo.
- Salvar estes dados

In [63]:
# Diretório onde estão os arquivos CSV
diretorio_air_cia = './data_engineer_test/AIR_CIA/'

In [64]:
# Normalizando as colunas para snake_case
nomes_colunas_air_cia = ['razao_social', 'icao_iata', 'cnpj', 'atividades_aereas',
                         'endereco_sede', 'telefone', 'email', 'decisao_operacional', 
                         'data_decisao_operacional', 'validade_operacional']

In [65]:
# Inicializa um DataFrame vazio para armazenar os dados
df_air_cia = pd.DataFrame()

In [66]:
# Liste todos os arquivos CSV na pasta
for filename in os.listdir(diretorio_air_cia):
    if filename.endswith(".csv"):
        # Crie o caminho completo para o arquivo
        arquivo = os.path.join(diretorio_air_cia, filename)
        
        # Leia o arquivo CSV e adicione os dados ao DataFrame total
        df = pd.read_csv(arquivo, sep=';')
        df_air_cia = pd.concat([df_air_cia, df], ignore_index=True)

In [67]:
# Modificando nome das colunas do dataframe AIR_CIA
df_air_cia.columns = nomes_colunas_air_cia


In [68]:
# separando os valores pelo espaço em icao_iata e colocando nas novas colunas: icao e iata
df_air_cia[['icao', 'iata']] = df_air_cia['icao_iata'].str.split(n=1, expand=True)

In [69]:
# Removendo icao_iata e deixando as novas colunas na mesma posição que ela estava 
posicao_icao_iata = nomes_colunas_air_cia.index('icao_iata')
nomes_colunas_air_cia.pop(posicao_icao_iata)
nomes_colunas_air_cia.insert(posicao_icao_iata, 'icao')
nomes_colunas_air_cia.insert(posicao_icao_iata + 1, 'iata')
df_air_cia = df_air_cia[nomes_colunas_air_cia]

In [132]:
print(df_air_cia.head(5))
print(df_air_cia.shape)

                                        razao_social icao  iata  \
0  AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...  ASO    2S   
1                        ATA - AEROTÁXI ABAETÉ LTDA.  ABJ  None   
2                               OMNI TÁXI AÉREO S.A.  OMI    O1   
3                                PRIME TÁXI AIR LTDA  NaN   NaN   
4                        SIDERAL LINHAS AÉREAS LTDA.  SID    0S   

                 cnpj                                  atividades_aereas  \
0  27.315.694/0001-02  SERVIÇOS AÉREOS PÚBLICOS, TÁXI-AÉREO, TRANSPOR...   
1  14.674.451/0001-19  TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, TRAN...   
2  03.670.763/0001-38  AEROFOTOGRAFIA, AEROINSPEÇÃO, SERVIÇOS AÉREOS ...   
3  23.568.370/0001-25  TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, SERV...   
4  10.919.908/0001-57  TRANSPORTE AÉREO NÃO REGULAR, TRANSPORTE AÉREO...   

                                       endereco_sede  \
0  RODOVIA PR 218, KM 7, JARDIM UNIVERSITÁRIO, AE...   
1   PÇA. GAGO COUTINHO, S/N,

In [133]:
df_air_cia = df_air_cia.drop_duplicates()

df_air_cia.to_csv('./data_export/AIR_CIA/air_cia_data.csv', index=False)

## Criar nova tabela aerodromos
- Através da API https://rapidapi.com/Active-api/api/airport-info/ trazer os aeródramos através do código ICAO presente nos dados de VRA.
- Salvar estes dados


In [105]:
# Definindo URL e headers para utilizar a API
url = 'https://airport-info.p.rapidapi.com/airport'

query_string = {"icao":"value"}

headers = {
	"X-RapidAPI-Key": "a593951eafmsh253dc4c7ca30e1cp1eec96jsnaef5fd9a9c11",
	"X-RapidAPI-Host": "airport-info.p.rapidapi.com"
}

In [106]:
# Coletando os ICAO únicos
icao_unicos = pd.concat([vra_df['icao_aerodromo_origem'], vra_df['icao_aerodromo_destino']]).unique()

In [121]:
len(icao_unicos)

315

In [117]:
data = pd.DataFrame([])

In [120]:
for icao in icao_unicos:
    query_string['icao'] = icao
#     print(query_string['icao'])
    response = requests.get(url, headers=headers, params=query_string)
    
    if response.status_code == 200:
        if 'error' in response.json():
            pass
        else:
            temp_data = pd.DataFrame([response.json()])
            data = pd.concat([data, temp_data], ignore_index=True)
    print(data)
    print()

      id iata  icao                                               name  \
0   6679  SDU  SBRJ                              Santos Dumont Airport   
1   2787  GYN  SBGO                             Santa Genoveva Airport   
2    953  BSB  SBBR  Brasília International Airport (Presidente J. ...   
3   6335  REC  SBRF  Recife/Guararapes–Gilberto Freyre Internationa...   
4   2320  FLN  SBFL                 Hercílio Luz International Airport   
5   7941  VCP  SBKP           Viracopos/Campinas International Airport   
6   6182  PVH  SBPV  Governador Jorge Teixeira de Oliveira Internat...   
7   3473  JPA  SBJP      Presidente Castro Pinto International Airport   
8   4500  MAO  SBEG                Eduardo Gomes International Airport   
9   6029  POA  SBPA                Salgado Filho International Airport   
10  8988  NAT  SBSG  São Gonçalo do Amarante–Governador Aluízio Alv...   
11  2695  GRU  SBGR          São Paulo–Guarulhos International Airport   
12  8322  XAP  SBCH                   

      id iata  icao                                               name  \
0   6679  SDU  SBRJ                              Santos Dumont Airport   
1   2787  GYN  SBGO                             Santa Genoveva Airport   
2    953  BSB  SBBR  Brasília International Airport (Presidente J. ...   
3   6335  REC  SBRF  Recife/Guararapes–Gilberto Freyre Internationa...   
4   2320  FLN  SBFL                 Hercílio Luz International Airport   
5   7941  VCP  SBKP           Viracopos/Campinas International Airport   
6   6182  PVH  SBPV  Governador Jorge Teixeira de Oliveira Internat...   
7   3473  JPA  SBJP      Presidente Castro Pinto International Airport   
8   4500  MAO  SBEG                Eduardo Gomes International Airport   
9   6029  POA  SBPA                Salgado Filho International Airport   
10  8988  NAT  SBSG  São Gonçalo do Amarante–Governador Aluízio Alv...   
11  2695  GRU  SBGR          São Paulo–Guarulhos International Airport   
12  8322  XAP  SBCH                   

      id iata  icao                                               name  \
0   6679  SDU  SBRJ                              Santos Dumont Airport   
1   2787  GYN  SBGO                             Santa Genoveva Airport   
2    953  BSB  SBBR  Brasília International Airport (Presidente J. ...   
3   6335  REC  SBRF  Recife/Guararapes–Gilberto Freyre Internationa...   
4   2320  FLN  SBFL                 Hercílio Luz International Airport   
5   7941  VCP  SBKP           Viracopos/Campinas International Airport   
6   6182  PVH  SBPV  Governador Jorge Teixeira de Oliveira Internat...   
7   3473  JPA  SBJP      Presidente Castro Pinto International Airport   
8   4500  MAO  SBEG                Eduardo Gomes International Airport   
9   6029  POA  SBPA                Salgado Filho International Airport   
10  8988  NAT  SBSG  São Gonçalo do Amarante–Governador Aluízio Alv...   
11  2695  GRU  SBGR          São Paulo–Guarulhos International Airport   
12  8322  XAP  SBCH                   

      id iata  icao                                               name  \
0   6679  SDU  SBRJ                              Santos Dumont Airport   
1   2787  GYN  SBGO                             Santa Genoveva Airport   
2    953  BSB  SBBR  Brasília International Airport (Presidente J. ...   
3   6335  REC  SBRF  Recife/Guararapes–Gilberto Freyre Internationa...   
4   2320  FLN  SBFL                 Hercílio Luz International Airport   
5   7941  VCP  SBKP           Viracopos/Campinas International Airport   
6   6182  PVH  SBPV  Governador Jorge Teixeira de Oliveira Internat...   
7   3473  JPA  SBJP      Presidente Castro Pinto International Airport   
8   4500  MAO  SBEG                Eduardo Gomes International Airport   
9   6029  POA  SBPA                Salgado Filho International Airport   
10  8988  NAT  SBSG  São Gonçalo do Amarante–Governador Aluízio Alv...   
11  2695  GRU  SBGR          São Paulo–Guarulhos International Airport   
12  8322  XAP  SBCH                   

      id iata  icao                                               name  \
0   6679  SDU  SBRJ                              Santos Dumont Airport   
1   2787  GYN  SBGO                             Santa Genoveva Airport   
2    953  BSB  SBBR  Brasília International Airport (Presidente J. ...   
3   6335  REC  SBRF  Recife/Guararapes–Gilberto Freyre Internationa...   
4   2320  FLN  SBFL                 Hercílio Luz International Airport   
5   7941  VCP  SBKP           Viracopos/Campinas International Airport   
6   6182  PVH  SBPV  Governador Jorge Teixeira de Oliveira Internat...   
7   3473  JPA  SBJP      Presidente Castro Pinto International Airport   
8   4500  MAO  SBEG                Eduardo Gomes International Airport   
9   6029  POA  SBPA                Salgado Filho International Airport   
10  8988  NAT  SBSG  São Gonçalo do Amarante–Governador Aluízio Alv...   
11  2695  GRU  SBGR          São Paulo–Guarulhos International Airport   
12  8322  XAP  SBCH                   

...

       id iata  icao                                               name  \
0    6679  SDU  SBRJ                              Santos Dumont Airport   
1    2787  GYN  SBGO                             Santa Genoveva Airport   
2     953  BSB  SBBR  Brasília International Airport (Presidente J. ...   
3    6335  REC  SBRF  Recife/Guararapes–Gilberto Freyre Internationa...   
4    2320  FLN  SBFL                 Hercílio Luz International Airport   
..    ...  ...   ...                                                ...   
281  6484  ROS  SAAR     Rosario – Islas Malvinas International Airport   
282  3531  KAN  DNKN            Mallam Aminu Kano International Airport   
283  7053  STL  KSTL            Lambert–St. Louis International Airport   
284  7913  VAG  SBVG                Major Brigadeiro Trompowsky Airport   
285  5365  NRT  RJAA                       Narita International Airport   

                                   location street_number  \
0    Rio de Janeiro, Rio de Janeiro, B

       id iata  icao                                               name  \
0    6679  SDU  SBRJ                              Santos Dumont Airport   
1    2787  GYN  SBGO                             Santa Genoveva Airport   
2     953  BSB  SBBR  Brasília International Airport (Presidente J. ...   
3    6335  REC  SBRF  Recife/Guararapes–Gilberto Freyre Internationa...   
4    2320  FLN  SBFL                 Hercílio Luz International Airport   
..    ...  ...   ...                                                ...   
283  7053  STL  KSTL            Lambert–St. Louis International Airport   
284  7913  VAG  SBVG                Major Brigadeiro Trompowsky Airport   
285  5365  NRT  RJAA                       Narita International Airport   
286   510  AZA  KIWA            Phoenix–Mesa Gateway Airport (FAA: IWA)   
287   628  BEG  LYBE                      Belgrade Nikola Tesla Airport   

                                   location street_number  \
0    Rio de Janeiro, Rio de Janeiro, B

       id iata  icao                                               name  \
0    6679  SDU  SBRJ                              Santos Dumont Airport   
1    2787  GYN  SBGO                             Santa Genoveva Airport   
2     953  BSB  SBBR  Brasília International Airport (Presidente J. ...   
3    6335  REC  SBRF  Recife/Guararapes–Gilberto Freyre Internationa...   
4    2320  FLN  SBFL                 Hercílio Luz International Airport   
..    ...  ...   ...                                                ...   
285  5365  NRT  RJAA                       Narita International Airport   
286   510  AZA  KIWA            Phoenix–Mesa Gateway Airport (FAA: IWA)   
287   628  BEG  LYBE                      Belgrade Nikola Tesla Airport   
288   280  ANF  SCFA                 Cerro Moreno International Airport   
289  1600  CVG  KCVG  Cincinnati/Northern Kentucky International Air...   

                                              location street_number  \
0               Rio de Jane

       id iata  icao                                               name  \
0    6679  SDU  SBRJ                              Santos Dumont Airport   
1    2787  GYN  SBGO                             Santa Genoveva Airport   
2     953  BSB  SBBR  Brasília International Airport (Presidente J. ...   
3    6335  REC  SBRF  Recife/Guararapes–Gilberto Freyre Internationa...   
4    2320  FLN  SBFL                 Hercílio Luz International Airport   
..    ...  ...   ...                                                ...   
287   628  BEG  LYBE                      Belgrade Nikola Tesla Airport   
288   280  ANF  SCFA                 Cerro Moreno International Airport   
289  1600  CVG  KCVG  Cincinnati/Northern Kentucky International Air...   
290   337  APX  SSOG                                  Arapongas Airport   
291  1427  CND  LRCK          Mihail Kogălniceanu International Airport   

                                              location street_number  \
0               Rio de Jane

       id iata  icao                                               name  \
0    6679  SDU  SBRJ                              Santos Dumont Airport   
1    2787  GYN  SBGO                             Santa Genoveva Airport   
2     953  BSB  SBBR  Brasília International Airport (Presidente J. ...   
3    6335  REC  SBRF  Recife/Guararapes–Gilberto Freyre Internationa...   
4    2320  FLN  SBFL                 Hercílio Luz International Airport   
..    ...  ...   ...                                                ...   
289  1600  CVG  KCVG  Cincinnati/Northern Kentucky International Air...   
290   337  APX  SSOG                                  Arapongas Airport   
291  1427  CND  LRCK          Mihail Kogălniceanu International Airport   
292  2298  FIH  FZAA           N'djili Airport (Kinshasa Int'l Airport)   
293  4870  MPM  FQMA                       Maputo International Airport   

                                              location street_number  \
0               Rio de Jane